# Titanic Survival Prediction — Project Summary

## What is This Project About?

We're building a **classification model** to predict whether a passenger on the Titanic survived or not, based on their characteristics:
- **Features:** Passenger class, Sex, Age, Number of siblings/spouses, Number of parents/children, Fare paid, Port of embarkation
- **Target:** Survived (1) or Did Not Survive (0)

This is a **binary classification problem** (two outcomes: yes or no).

---

## Data Pipeline

```
Raw Data
   ↓
[Data Preprocessing]
   ├─ Numerical Features (Age, Fare, etc.) → Median Imputation
   └─ Categorical Features (Sex, Embarked, etc.) → Mode Imputation + One-Hot Encoding
   ↓
[Combined Features]
   ↓
[Train Models & Evaluate]
```

---

## Model 1: Support Vector Classifier (SVC)

### How SVC Works

SVC finds the **best decision boundary** that separates "Survived" from "Did Not Survive" passengers.

**Think of it like drawing a line on a map:**
- One side of the line = Survived (1)
- Other side of the line = Did Not Survive (0)
- The goal: Maximize the distance between the line and the data points

### RBF Kernel — Handling Complex Patterns

```python
SVC(gamma="auto", random_state=42)
```

**Kernel:** Determines how SVC draws the decision boundary
- **Linear Kernel:** Straight line (for linearly separable data)
- **RBF (Radial Basis Function) Kernel:** Curved, complex boundaries (for non-linear patterns)

We use **RBF** because survival data is complex — a straight line won't separate "survived" from "did not survive" well enough.

**Visual example:**

```
Linear Kernel (simple)     RBF Kernel (complex)
━━━━━━━━━━━━━━━━━━━━━━━     ╱╲╱╲╱╲╱╲╱╲╱╲
│  ○ ○ ○                     │ ○ ╲ ○ ╲ ○
│ ─ ─ ─ ─ (line)            ├───●───●─── (wavy curve)
│  × × ×                     │ × ╱ × ╱ ×
```

### Gamma Parameter — How "Local" the Decision is

**Gamma = "auto"** means sklearn automatically calculates it as `1 / (n_features)`

**Gamma controls:** How far the influence of each training example reaches
- **Low Gamma (≤ 0.01):** Each example has **far-reaching influence** → Simple, smooth boundaries → Might underfit
- **High Gamma (≥ 0.1):** Each example has **local influence only** → Complex, wiggly boundaries → Might overfit
- **"auto":** Usually gives a good balance

**Visual example:**

```
Low Gamma                  High Gamma
(smooth boundary)          (complex boundary)
    ╱─────╲                ╱╲╱╲╱╲╱╲
   │ ○ ○ ○ │              │○╲ ○╱○╲
───┤   ┌──┐ ├───           ├─●─●─●─┤
   │ × × × │              │×╱ ×╲×╱
    ╲─────╱                ╲╱╲╱╲╱╲╱
```

### SVC Results

```python
svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
# Mean accuracy: ~78-80%
```

✅ **Good start**, but can we do better?

---

## Model 2: Random Forest Classifier

### How Random Forest Works

**Random Forest = "Forest" of Decision Trees**

Instead of one tree, we grow **100 decision trees** (n_estimators=100), each trained on a **random subset** of data:

```python
RandomForestClassifier(n_estimators=100, random_state=42)
```

**How it makes predictions:**
1. Each tree makes its own prediction (Survived or Did Not Survive)
2. **Vote:** All 100 trees vote
3. **Winner:** Prediction with the most votes wins

**Example with 3 trees:**
```
Tree 1 says: "Survived"  ✓
Tree 2 says: "Survived"  ✓
Tree 3 says: "Did Not"   ✗
            ───────
Result: "Survived" (2 out of 3 votes)
```

### Why Random Forest is Better for This Dataset

| Aspect | SVC | Random Forest |
|--------|-----|---------------|
| **Handles non-linear patterns** | Good (with RBF kernel) | Excellent (naturally) |
| **Handles mixed data types** | Needs preprocessing | Handles easily |
| **Interactions between features** | Struggles | Naturally captures |
| **Interpretability** | "Black box" | Can see feature importance |
| **Robustness** | Sensitive to outliers | Less sensitive (voting) |
| **Speed** | Fast training | Slower but worth it |

### Random Forest Results

```python
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
# Mean accuracy: ~81-83%
```

**Random Forest performs better!** 🎉

**Why?** The Titanic dataset has complex relationships:
- Women had higher survival rates
- Higher passenger class → Higher survival rates
- Age mattered (children vs. adults)
- Fare amount (wealth) mattered

These **non-linear interactions** are exactly what Random Forest excels at.

---

## Comparison

```
Model Accuracy Comparison
│
│ 82% ─────────────────●──────── Random Forest (WINNER)
│ 80% ──────────●────────────── SVC
│ 78% ─────────────────────────
└─────────────────────────────
    SVC        Random Forest
```

**Lesson:** Different models work better for different problems. **Always try multiple models!**

---

## Key Takeaways

1. **SVC with RBF kernel** is powerful but requires careful tuning (gamma, C parameters)
2. **Random Forest** is more flexible and often works better with real-world, messy data
3. **Cross-validation** gives us honest performance estimates (not overly optimistic)
4. **Feature engineering** (preprocessing pipelines) is crucial for both models
5. **Always compare models** before choosing one for production

In [ ]:
import pandas as pd
import os

In [ ]:
# Loading the dataset
# Download dataset: https://www.kaggle.com/c/titanic/data
TITANIC_PATH = '../datasets/titanic'
def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [ ]:
#Load the data. Data is divided into train and test sets. We will use the train set to train our model and the test set to evaluate its performance.
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")
train_data.head()

In [ ]:
# Explore the data to understand its structure and identify any missing values or outliers. This will help us decide how to preprocess the data before training our model.
train_data.info()
# 3 attributes have missing values

In [ ]:
train_data.describe()

In [ ]:
# Coount of female passengers
train_data["Sex"].value_counts()["female"]

In [ ]:
# Creating a custom transformer to select specific columns from the DataFrame. This is useful when we want to apply different preprocessing steps to different subsets of features.
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# Creating a pipeline for preprocessing the numerical attributes. This pipeline will first select the numerical attributes, then impute any missing values using the median strategy.
num_pipeline = Pipeline([
    ("select_numeric", DataFrameSelector(["Age", "SibSp", "Parch", "Fare"])),
    ("imputer", SimpleImputer(strategy="median")),
])

In [ ]:
num_pipeline.fit_transform(train_data)

In [ ]:
# Creating a custom transformer to impute missing values in categorical attributes with the most frequent value. This is useful when we want to handle missing values in categorical features before training our model.
#Suppose sex column has missing values, we can impute them with the most frequent value in that column.Like if most of the values in the sex column are male, then we can impute the missing values with male.
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Compute the most frequent value for each column and store it in an attribute called most_frequent_. This will be used later in the transform method to fill the missing values.
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["Pclass", "Sex", "Embarked"])),
        ("imputer", MostFrequentImputer()),
        ("cat_encoder", OneHotEncoder()),
    ])

In [ ]:
# Combining the numerical and categorical pipelines into a single pipeline using FeatureUnion. This allows us to apply different preprocessing steps to different subsets of features and then combine the results into a single feature matrix that can be used to train our model.
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])


In [ ]:
# Seperating the features and the target variable. The target variable is "Survived", which indicates whether a passenger survived or not. The features are all the other columns in the dataset.
X_train = preprocess_pipeline.fit_transform(train_data)
y_train = train_data["Survived"]

In [ ]:
#Train SVC Classifier
from sklearn.svm import SVC
svm_clf  = SVC(gamma="auto", random_state=42)
svm_clf.fit(X_train, y_train)

In [ ]:
X_test = preprocess_pipeline.transform(test_data)
y_pred = svm_clf.predict(X_test)

In [ ]:
# Evaluate model
from sklearn.model_selection import cross_val_score

# Cross-Validation Score — What It Tells Us and How It's Computed

## What the Score Tells Us

The score represents **accuracy** (for classification tasks like Titanic survival prediction):
- **Range:** 0 to 1 (or 0% to 100%)
- **0.85 means:** The model correctly predicts survival for 85% of passengers
- **Returns:** An array of scores from each fold

---

## How It's Computed (K-Fold Cross-Validation)

With `cv=10`, the data is split into **10 folds** (10 parts):

```
Original Dataset
│
├─ Fold 1: Test on fold 1, Train on folds 2-10 → Score 1
├─ Fold 2: Test on fold 2, Train on folds 1,3-10 → Score 2
├─ Fold 3: Test on fold 3, Train on folds 1-2,4-10 → Score 3
...
└─ Fold 10: Test on fold 10, Train on folds 1-9 → Score 10

Result: [score1, score2, score3, ..., score10]
```

**Process for each fold:**
1. **Train** the model on 9 folds (90% of data)
2. **Test** the model on 1 fold (10% of data)
3. **Record** the accuracy score

---

## Why Use Cross-Validation Instead of Train/Test Split?

| Approach | Benefit | Drawback |
|---|---|---|
| **Single Train/Test Split** | Simple, fast | High variance (depends on which examples in test set) |
| **Cross-Validation (cv=10)** | Reliable, uses all data for training | Slower (trains 10 times) |

**Cross-validation solves the variance problem:**
- You get 10 scores → You see the **range and average**
- More reliable estimate of real-world performance

---

## Interpreting Cross-Validation Results

```python
svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
# Output: array([0.82, 0.85, 0.88, 0.80, 0.86, 0.84, 0.87, 0.83, 0.85, 0.81])

print(f"Mean: {svm_scores.mean():.3f}")  # 0.841 (average accuracy)
print(f"Std:  {svm_scores.std():.3f}")   # 0.024 (how much scores vary)
```

**Interpretation:**
- ✅ Your SVM model has ~84.1% average accuracy
- ✅ The ±2.4% variation is small → model is stable
- ✅ Not overfitting (scores are consistent across folds)

---

## Common cv Values

| cv | Use Case |
|---|---|
| **5** | Standard, good balance of speed/reliability |
| **10** | More thorough, slower |
| **LeaveOneOut** | Maximum accuracy but very slow |
| **StratifiedKFold** | Better for imbalanced classes |

In [ ]:
svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
svm_scores.mean()

In [ ]:
#Check using Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()
#Random Forest Classifier performs better than SVC Classifier in this case.
